In [0]:
import sys

In [0]:
def reduce_Op(formula) :
    if (isinstance(formula, str)) :
        return formula
    op = formula[0]
    temp = []
    p = []
    for i, x in enumerate(formula) :
        if (i > 0) :
            if (isinstance(x, str)) :
                temp.append(x)
            elif (isinstance(x, list)) :
                p.append(reduce_Op(x))
    newFormula = temp
    for x in p :
        if (isinstance(x, list) and x[0] == op) :
            for i,clause in enumerate(x) :
                if (i > 0) :
                    newFormula.append(clause)
        else :
            newFormula.append(x)
    newFormula.insert(0, op)
    return newFormula

In [0]:
def rem_Duplicates(formula) :
    if (isinstance(formula, str) or (isinstance(formula, list) and formula[0] == "not" and isinstance(formula[1], str))) :
        return formula    
    for i, checkItem in enumerate(formula) :
        if (i > 0) :
            for j, x in reversed(list(enumerate(formula))) :
                if (j > i) :
                    if (isinstance(x, list)) :
                        newItem = rem_Duplicates(x)
                        formula.insert(j, newItem)
                        formula.remove(x)
                    if (checkItem == x) :
                        formula.remove(x)
    if (isinstance(formula, list) and formula[0] != "not" and len(formula) < 3) :
        return formula[1]
    return formula

In [0]:
def sort(formula) :
    if (isinstance(formula, str)) :
        return formula
    op = formula[0]
    if (op == "implies") :
        return formula
    temp = []
    p = []
    for i, x in enumerate(formula) :
        if (i > 0) :
            if (isinstance(x, str)) :
                temp.append(x)
            elif (isinstance(x, list)) :
                p.append(sort(x))
    if (len(temp) > 0) :
        temp.sort()
    if (len(p) > 0) :
        p = sorted(p, key=lambda proposition: proposition[0], reverse=True)
    newFormula = temp + p
    newFormula.insert(0, op)
    return newFormula

In [0]:
def convert(formula) :
    if (isinstance(formula, str)):
        return formula
    elif(isinstance(formula,list)) :
        if (formula[0] == "implies") :
            return convert(["or", convert(["not",convert(formula[1])]), convert(formula[2])])
        elif (formula[0] == "iff") :
            return convert(["and", convert(["or", convert(["not", formula[1]]), formula[2]]), convert(["or", formula[1], convert(["not", formula[2]])])])
        elif (formula[0] == "not") :
            if (isinstance(formula[1], str)) :
                return formula
            elif (isinstance(formula[1], list) and (formula[1])[0] == "not") :
                return convert((formula[1])[1])
            elif (isinstance(formula[1], list) and (formula[1])[0] == "and") :
                disjuncts = []
                for i, x in enumerate(formula[1]) :
                    if (i > 0) :
                        disjuncts.append(convert(["not", x]))
                disjuncts.insert(0, "or")
                return convert(disjuncts)
            elif (isinstance(formula[1], list) and (formula[1])[0] == "or") :
                conjuncts = []
                for i, x in enumerate(formula[1]) :
                    if (i > 0) :
                        conjuncts.append(convert(["not",]))
                conjuncts.insert(0, "and")
                return convert(conjuncts)
            elif (isinstance(formula[1], list) and ((formula[1])[0] == "implies")) :
                return convert(["and", convert((formula[1])[1]), ["not", convert((formula[1][2]))]])
            elif (isinstance(formula[1], list) and (formula[1])[0] == "iff") :
                return convert(["not", convert(formula[1])])
        elif (formula[0] == "or") :
            formula = sort(formula)
            formula = rem_Duplicates(formula)
            formula = reduce_Op(formula)
            formula = sort(formula)
            if (len(formula) == 1) :
                return formula
            if ((isinstance(formula[-1], list) and (formula[-1])[0] == "and")) :
                conjuncts = []
                for i, x in enumerate(formula[-1]) :
                    if (i > 0) :
                        conjuncts.append(["or", formula[-2], x])
                conjuncts.insert(0, "and")
                if (len(formula) < 4) :
                    return convert(conjuncts)
                else :
                    formula.remove(formula[-1])
                    formula.remove(formula[-1])
                formula.append(conjuncts)
                return convert(formula)
            elif ((isinstance(formula[1], str) and isinstance(formula[2], str)) or (isinstance(formula[1], str) and isinstance(formula[2], list) and (formula[2])[0] == "not" and isinstance((formula[2])[1], str)) or (isinstance(formula[2], str) and isinstance(formula[1], list) and (formula[1])[0] == "not" and isinstance((formula[1])[1], str))) :
                formula.append(["or", formula[1], formula[2]])
                formula.remove(formula[1])
                formula.remove(formula[1])
                
                formula = reduce_Op(formula)
                formula = sort(formula)
                return formula
            elif (isinstance(formula[1], list) and (formula[1])[0] == "not" and isinstance((formula[1])[1], str) and isinstance(formula[2], list) and (formula[2])[0] == "not" and isinstance((formula[2])[1], str)) :
                return formula
            else :
                disjuncts = []
                for i, x in enumerate(formula) :
                    if (i > 0) :
                        disjuncts.append(convert(x))
                disjuncts.insert(0, "or")
                return convert(disjuncts)
            
        elif (formula[0] == "and") :
            formula = sort(formula)
            formula = rem_Duplicates(formula)
            formula = reduce_Op(formula)
            formula = sort(formula)
            if (len(formula) == 1) :
                return formula
            disjuncts = []
            for i, x in enumerate(formula) :
                if (i > 0) :
                    disjuncts.append(convert(x))
            disjuncts.insert(0, "and")
            
            disjuncts = reduce_Op(disjuncts)
            return disjuncts        

In [12]:
convert(["and", "A", ["or", "C", "D"], ["or", "D", "C"]])

['and', 'A', ['or', 'C', 'D']]